In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig
import pandas_ta as ta

In [2]:
df = pd.read_csv("../../db/data/merge/tertiary/idxs_.csv",index_col=0)
df_logs = pd.read_csv("../../db/data/merge/tertiary/logs_.csv",index_col=0)
h = pd.read_csv("../../db/data/merge/tertiary/higs_.csv",index_col=0)
l = pd.read_csv("../../db/data/merge/tertiary/lows_.csv",index_col=0)


## *Bollinger Bands Function* 
#### (we are not going to use this function, just to "show off")

In [3]:
def get_bollinger_bands(df, rate=5, sigma_start=2, sigma_stop=2):
    
    sma = df.rolling(rate).mean()
    bollinger_bands = {"sma":sma}
    std = df.rolling(rate).std()
    for sigma in range(sigma_start, sigma_stop+1):
        up = sma + std * sigma
        down = sma - std * sigma
        bollinger_bands[f'up{sigma}'] = up
        bollinger_bands[f'down{sigma}'] = down
    bol = pd.DataFrame.from_dict(bollinger_bands)
    bol.set_index(df.index, inplace=True)
    return bol


## *Bollinger reduced*

In [4]:
def bollinger_small(df, rate=5, sigma_start=2, sigma_stop=2):
    
    data = pd.DataFrame([])
    
    for currency in df.columns:
        data[f'{currency}_sma'] = df[currency].rolling(rate).mean()
        data[f'{currency}_std']= df[currency].rolling(rate).std()
        
    data.index = df.index
    
    return data

## *Volatility Function*

In [5]:
def volatility(df,rate=240,window=506):
    
    data = pd.DataFrame([])
    for currency in df.columns:
        data[f"{currency}_vol1"] = df[currency].rolling(rate).std()/(window**0.5)
        data[f"{currency}_vol2"] = df[currency].rolling(rate).std()*(window**0.5)
    data.index = df.index
    
    return data

## *ATR Function*

In [6]:

# def atr(close: pd.DataFrame, low: pd.DataFrame, high: pd.DataFrame, window:int=14):
#     data = pd.DataFrame([])
#     ccy_pairs = close.columns
#     for pair in ccy_pairs:
#         data[f'close_{pair}'] = close[pair]
#         data[f'high_{pair}'] = high[pair]
#         data[f'low_{pair}'] = low[pair]
#         data[f'tr0_{pair}'] = data[f'high_{pair}'] - data[f'low_{pair}']
#         data[f'tr1_{pair}'] = np.abs(data[f'high_{pair}'] - data[f'close_{pair}'].shift())
#         data[f'tr2_{pair}'] = np.abs(data[f'low_{pair}'] - data[f'close_{pair}'].shift())
#         data[f'tr_{pair}'] = data[[f'tr0_{pair}', f'tr1_{pair}', f'tr2_{pair}']].max(axis=1)
#         data[f'atr_{pair}'] = ema(data[f'tr_{pair}'], window)
#     data.dropna(inplace=True)
#     return datab

In [7]:
def ema(df: pd.Series, window: int):
    return df.ewm(alpha=1/window, adjust=False).mean()
def atr(idxs: pd.DataFrame, low: pd.DataFrame, high: pd.DataFrame, window:int=14):
    data = pd.DataFrame([])
    for currency in idxs.columns:
        data[f'idxs_{currency}'] = idxs[currency]
        data[f'high_{currency}'] = high[currency]
        data[f'low_{currency}'] = low[currency]
        data[f'tr0_{currency}'] = data[f'high_{currency}'] - data[f'low_{currency}']
        data[f'tr1_{currency}'] = np.abs(data[f'high_{currency}'] - data[f'idxs_{currency}'].shift())
        data[f'tr2_{currency}'] = np.abs(data[f'low_{currency}'] - data[f'idxs_{currency}'].shift())
        data[f'tr_{currency}'] = data[[f'tr0_{currency}', f'tr1_{currency}', f'tr2_{currency}']].max(axis=1)
        data[f'atr_{currency}'] = ema(data[f'tr_{currency}'], window)
        data.drop(columns=[f'idxs_{currency}',f'tr0_{currency}',f'tr1_{currency}',f'tr2_{currency}',f'tr_{currency}'], inplace=True)
    return data

In [8]:
atr = atr(idxs=df,low=l,high=h)
atr

,high_AUD,low_AUD,atr_AUD,high_CAD,low_CAD,atr_CAD,high_CHF,low_CHF,atr_CHF,high_EUR,...,atr_GBP,high_JPY,low_JPY,atr_JPY,high_NZD,low_NZD,atr_NZD,high_USD,low_USD,atr_USD
2010-01-04 00:00:00+00:00,0.098196,0.121660,-0.023465,0.085321,0.133467,-0.048146,0.136353,0.163080,-0.026727,0.121904,...,-0.068512,0.148129,0.093168,0.054961,0.150415,0.133749,0.016666,0.128846,0.107970,0.020876
2010-01-04 01:00:00+00:00,0.102392,0.089160,7.114700,0.157696,0.083594,7.092179,0.060113,0.093569,7.113745,0.066515,...,7.076556,0.107555,0.066792,7.189121,0.128831,0.087580,7.152077,0.130043,0.075092,7.156879
2010-01-04 02:00:00+00:00,0.092015,0.084737,13.740927,0.218213,0.074254,13.732089,0.091950,0.086290,13.738733,0.060717,...,13.702947,0.059279,0.110547,13.816052,0.143595,0.106146,13.769635,0.124149,0.076842,13.791264
2010-01-04 03:00:00+00:00,0.077562,0.089172,19.892070,0.078039,0.114778,19.915477,0.081172,0.061485,19.895638,0.084415,...,19.855706,0.174011,0.077016,19.965222,0.093720,0.098518,19.911645,0.078866,0.093789,19.940470
2010-01-04 04:00:00+00:00,0.046955,0.048731,25.609811,0.049811,0.037763,25.651970,0.037296,0.051123,25.615135,0.046643,...,25.562097,0.061329,0.039604,25.689036,0.059331,0.055738,25.618017,0.049608,0.044580,25.655073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07 09:00:00+00:00,0.059448,0.109826,102.409296,0.073281,0.077009,104.502804,0.038942,0.168481,102.316544,0.093107,...,95.406430,0.040206,0.121793,87.753330,0.055669,0.104861,99.629075,0.056465,0.122416,108.827522
2022-09-07 10:00:00+00:00,0.117255,0.073690,102.412332,0.117155,0.094288,104.511753,0.090165,0.097785,102.320829,0.050054,...,95.401073,0.073228,0.221882,87.709541,0.123344,0.061447,99.637063,0.140251,0.102688,108.836615
2022-09-07 11:00:00+00:00,0.131591,0.067429,102.415484,0.107459,0.074807,104.532011,0.121490,0.065370,102.328634,0.112727,...,95.380899,0.118971,0.089326,87.654524,0.141303,0.069569,99.641383,0.150749,0.076997,108.862024
2022-09-07 12:00:00+00:00,0.036495,0.091789,102.411656,0.060946,0.139458,104.556185,0.068564,0.067451,102.342297,0.047668,...,95.363958,0.053505,0.137683,87.605076,0.040063,0.095190,99.637645,0.049400,0.118520,108.900481


## *Sharpe Ratio*

In [9]:
def sharpe_ratio(df,rate=24):

    data = pd.DataFrame([])
    for currency in df.columns:
        data[f'{currency}_sharpe'] = df[currency].rolling(rate).mean()/df[currency].rolling(rate).std()
    data.index = df.index

    return data

In [10]:
d = sharpe_ratio(df_logs)

In [11]:
d.iloc[200:300]

,AUD_sharpe,CAD_sharpe,CHF_sharpe,EUR_sharpe,GBP_sharpe,JPY_sharpe,NZD_sharpe,USD_sharpe
2010-01-14 08:00:00+00:00,0.145086,0.206655,-0.033919,0.003042,0.126108,-0.239548,0.000545,-0.156489
2010-01-14 09:00:00+00:00,0.118997,0.227897,-0.112434,-0.029151,0.089909,-0.195737,-0.013900,-0.059089
2010-01-14 10:00:00+00:00,0.200276,0.211110,-0.211857,-0.110996,0.073460,-0.266535,0.138979,-0.075098
2010-01-14 11:00:00+00:00,0.246345,0.205205,-0.295194,-0.207364,-0.016712,-0.164996,0.110580,0.021597
2010-01-14 12:00:00+00:00,0.170232,0.202504,-0.311933,-0.237068,0.017116,-0.081165,0.044418,0.095465
...,...,...,...,...,...,...,...,...
2010-01-20 07:00:00+00:00,-0.104508,0.015019,-0.280953,-0.312822,0.048242,0.220287,-0.272476,0.406326
2010-01-20 08:00:00+00:00,-0.057228,0.063968,-0.254109,-0.307707,0.123564,0.112482,-0.252521,0.370024
2010-01-20 09:00:00+00:00,-0.041215,0.039341,-0.147428,-0.233806,0.088189,0.159263,-0.313676,0.350739
2010-01-20 10:00:00+00:00,-0.030381,-0.093745,-0.067969,-0.118150,0.060348,0.231275,-0.368628,0.329085


In [12]:
def fractals(df):

    data = pd.DataFrame(index = df.index, columns=[x +"_fractal" for x in df.columns])
    
    for ccy in df.columns:
        
        for i in range(len(df.index)):
             
                if df[ccy].iloc[i-2] > df[ccy].iloc[i-1] and df[ccy].iloc[i-1] < df[ccy].iloc[i]:
                    data[f'{ccy}_fractal'][i] = 1
                elif df[ccy].iloc[i-2] < df[ccy].iloc[i-1] and df[ccy].iloc[i-1] > df[ccy].iloc[i]:
                    data[f'{ccy}_fractal'][i] = -1
                else:
                    data[f'{ccy}_fractal'][i] = 0
            
    data.fillna(0, inplace=True)
    data.index = df.index
    return data



In [13]:
f = fractals(df)
f

KeyboardInterrupt: 

In [ ]:
f.nunique()

bear_fractal    2
bull_fractal    2
dtype: int64

## Stocastic Adaptative indicator

In [14]:
def fibonacci(close, highs, lows, k_period, d_period):
    data = pd.DataFrame(index=highs.columns, inplace=True)
    for ccy in highs.columns:
        # Adds a "n_high" column with max value of previous 14 periods
        data['n_high'] = highs[ccy].rolling(k_period).max()
        #  Adds an "n_low" column with min value of previous 14 periods
        data['n_low'] = lows[ccy].rolling(k_period).min()
    # Uses the min/max values to calculate the %k (as a percentage)
        data['%K'] = (close[ccy] - data['n_low']) * 100 / (data['n_high'] - data['n_low'])
    # Uses the %k to calculates a SMA over the past 3 values of %k
        data['%D'] = data['%K'].rolling(d_period).mean()

    
# Add some indicators
    data.ta.stoch(high='high', low='low', k=14, d=3, append=True)
